## TensorFlow

In this lecture we fixed a number of parameters for the neural model, including:
* the number of layers
* the number of neurons per layer
* the batch size
* the number of epochs
* the optimizer

The choice of these parameters was pretty much randomean. There was no formal method we used to set these parameters. Perform a grid search (similar to the one we performed in our scikit-learn lectures) varying the range of these parameters within the following ranges/sets:

* number of layers [low = 3, high = 5, step = 1]
* number of neurons **per layer** [low = 32, high = 256, step = 32 ]
* batch size [low = 1000, high = 10000, step = 1000]
* number of epochs [low = 10, high = 30, step = 10]
* optimizer {Adam, SGD, AdamW}

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD, AdamW
import time

# Load the Fashion MNIST data
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Smaller subset of the data for demonstration/time reasons 
train_images, train_labels = train_images[:10000], train_labels[:10000]
test_images, test_labels   = test_images[:2000],   test_labels[:2000]

In [2]:
def build_model(num_layers=3, neurons=128, optimizer='adam'):
    """
    Build and compile a Sequential model with:
      - Flatten input,
      - `num_layers` hidden Dense layers, each with `neurons` units, ReLU activation,
      - Output layer with 10 units (softmax).
      - `optimizer` one of ['adam', 'sgd', 'adamw'].

    Returns compiled model.
    """
    model = Sequential()
    model.add(Input(shape=(28, 28)))
    model.add(Flatten())

    for _ in range(num_layers):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(10, activation='softmax'))
    
    # Create the optimizer
    if optimizer.lower() == 'adam':
        opt = Adam()
    elif optimizer.lower() == 'sgd':
        opt = SGD()
    elif optimizer.lower() == 'adamw':
        opt = AdamW()
    else:
        raise ValueError(f"Unknown optimizer: {optimizer}")

    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [3]:
num_layers_list = [3, 4, 5]
neurons_list = range(32, 257, 32)      # 32 to 256 (step=32)
batch_size_list = range(1000, 10001, 1000)  # 1000..10000 step=1000
epochs_list = [10, 20, 30]
optimizers_list = ['adam', 'sgd', 'adamw']


best_acc = 0.0
best_params = None

start_time = time.time()

for num_layers in num_layers_list:
    for neurons in neurons_list:
        for batch_size in batch_size_list:
            for epochs in epochs_list:
                for optimizer_name in optimizers_list:
                    
                    # 1. Build the model
                    model = build_model(
                        num_layers=num_layers,
                        neurons=neurons,
                        optimizer=optimizer_name
                    )
                    
                    # 2. Fit/Train
                    history = model.fit(
                        train_images, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0  # no output
                    )
                    
                    # 3. Evaluate on TEST set (but typically you'd do this on a validation set)
                    loss, acc = model.evaluate(test_images, test_labels, verbose=0)
                    
                    # 4. Check if we got a better accuracy
                    if acc > best_acc:
                        best_acc = acc
                        best_params = {
                            'num_layers': num_layers,
                            'neurons': neurons,
                            'batch_size': batch_size,
                            'epochs': epochs,
                            'optimizer': optimizer_name
                        }
                        
                        # Print interim result (optional, helps track progress)
                        print(f"New best acc={acc:.4f} with {best_params}")

end_time = time.time()
print(f"\nFinished grid search in {end_time - start_time:.2f} seconds.")
print("Best test accuracy:", best_acc)
print("Best parameters:", best_params)


New best acc=0.8045 with {'num_layers': 3, 'neurons': 32, 'batch_size': 1000, 'epochs': 10, 'optimizer': 'adam'}
New best acc=0.8395 with {'num_layers': 3, 'neurons': 32, 'batch_size': 1000, 'epochs': 20, 'optimizer': 'adam'}
New best acc=0.8445 with {'num_layers': 3, 'neurons': 32, 'batch_size': 1000, 'epochs': 20, 'optimizer': 'adamw'}
New best acc=0.8450 with {'num_layers': 3, 'neurons': 32, 'batch_size': 1000, 'epochs': 30, 'optimizer': 'adam'}
New best acc=0.8545 with {'num_layers': 3, 'neurons': 32, 'batch_size': 1000, 'epochs': 30, 'optimizer': 'adamw'}
New best acc=0.8585 with {'num_layers': 3, 'neurons': 64, 'batch_size': 1000, 'epochs': 20, 'optimizer': 'adamw'}
New best acc=0.8590 with {'num_layers': 3, 'neurons': 96, 'batch_size': 1000, 'epochs': 20, 'optimizer': 'adam'}
New best acc=0.8660 with {'num_layers': 3, 'neurons': 96, 'batch_size': 1000, 'epochs': 30, 'optimizer': 'adam'}
New best acc=0.8730 with {'num_layers': 3, 'neurons': 160, 'batch_size': 1000, 'epochs': 30, 

In [5]:
print(tf.config.list_physical_devices('CPU'))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
